In [1]:
import numpy as np
import pandas as pd
import pymc3 as pm

import matplotlib.pyplot as plt
%matplotlib inline 

import glam

%load_ext autoreload
%autoreload 2

/Users/felixmolter/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Systematic parameter recovery (Metropolis)

Here, we perform a structured and systematic parameter recovery study. We use the original model variant with 4 parameters ($v$, $\gamma$, $\sigma$, $\tau$). For each parameter, we define a range of sensible values and select a `low`, `medium` and `high` value, based on the individual parameter estimates we obtained by hierarchically fitting the GLAM to the data from Krajbich & Rangel (2011) in Thomas, Molter, Krajbich, Heekeren & Mohr (submitted).

We then pick one parameter, one constellation of values of the remaining three parameters (e.g., we pick $v$, and set $\gamma$, $\sigma$ and $\tau$ to their `low` values). Then, for 10 different values of $v$ along its range, we generate a synthetic dataset for a single participant with N=100 trials, random item values and gaze, fit the model and record the generating and estimated parameters. The procedure is repeated for all possible constellation of other parameter values, and other parameters, resulting in 1080 ($10 \times 4 \times 3^3$) simulated and fitted datasets total.

## Define parameter ranges from 2011 estimates

In [2]:
bounds = dict(v=[0.000015, 0.00015],
              gamma=[-1, 1],
              s=[0.004, 0.011],
              tau=[0.1, 1.25])

ranges = {parameter: np.linspace(*bounds[parameter], 10)
          for parameter in ['v', 'gamma', 's', 'tau']}
ranges

{'v': array([1.50e-05, 3.00e-05, 4.50e-05, 6.00e-05, 7.50e-05, 9.00e-05,
        1.05e-04, 1.20e-04, 1.35e-04, 1.50e-04]),
 'gamma': array([-1.        , -0.77777778, -0.55555556, -0.33333333, -0.11111111,
         0.11111111,  0.33333333,  0.55555556,  0.77777778,  1.        ]),
 's': array([0.004     , 0.00477778, 0.00555556, 0.00633333, 0.00711111,
        0.00788889, 0.00866667, 0.00944444, 0.01022222, 0.011     ]),
 'tau': array([0.1       , 0.22777778, 0.35555556, 0.48333333, 0.61111111,
        0.73888889, 0.86666667, 0.99444444, 1.12222222, 1.25      ])}

In [3]:
# Determine low, medium, high values as 10th, 50th and 90th percentile of these ranges
values = {parameter: np.percentile(np.linspace(*bounds[parameter], 100), [10, 50, 90])
          for parameter in ['v', 'gamma', 's', 'tau']}
values

{'v': array([2.850e-05, 8.250e-05, 1.365e-04]),
 'gamma': array([-8.00000000e-01,  5.55111512e-17,  8.00000000e-01]),
 's': array([0.0047, 0.0075, 0.0103]),
 'tau': array([0.215, 0.675, 1.135])}

# Recover a single GLAM

In [4]:
def recover_glam(parameters, n_trials=100, n_items=3, seed=None):

    G = glam.GLAM()

    G.simulate_group(kind='individual',
                     n_individuals=1,
                     n_trials=n_trials, 
                     n_items=n_items,
                     parameters=parameters,
                     error_weight=0,
                     value_range=[1, 10],
                     seed=seed)
    G.make_model('individual', t0_val=0, error_weight=0)
    G.fit(method='MCMC', draws=10000, step=pm.Metropolis, njobs=1, tune=1000, progressbar=False)
    
    return G

In [5]:
parameters = dict(v=[ranges['v'][0]],
                  gamma=[values['gamma'][0]],
                  s=[values['s'][0]],
                  tau=[values['tau'][0]],
                  t0=[0])

result = recover_glam(parameters=parameters, seed=1)

Generating single subject models for 1 subjects...
Fitting 1 model(s) using MCMC...
  Fitting model 1 of 1...


Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
The estimated number of effective samples is smaller than 200 for some parameters.


/!\ Automatically setting parameter precision...


# Run over multiple parameter sets

In [6]:
parameter_info = {parameter: dict(low=values[parameter][0],
                                  medium=values[parameter][1],
                                  high=values[parameter][2],
                                  variable=ranges[parameter])
                  for parameter in ['v', 'gamma', 's', 'tau']}

In [7]:
def generate_parameter_sets(parameter_info):

    from itertools import product
    
    levels = ['low', 'medium', 'high']
    parameters = ['v', 'gamma', 's', 'tau']
    
    constellations = list(product(levels, levels, levels))

    index = 0
    
    for variable_parameter in parameter_info.keys():
        
        others = [p for p in parameters
                  if p != variable_parameter]
        
        for other_constellation in constellations:

            for variable_value in parameter_info[variable_parameter]['variable']:

                level_set = dict()
                level_set[variable_parameter] = 'variable'
                parameter_set = dict(t0=[0])
                parameter_set[variable_parameter] = [variable_value]

                for o, other in enumerate(others):
                    level_set[other] = other_constellation[o]
                    parameter_set[other] = [parameter_info[other][other_constellation[o]]]
                    
                index += 1
                yield index, level_set, parameter_set

In [8]:
all_generated_inputs = list(generate_parameter_sets(parameter_info=parameter_info))
all_generated_inputs[:2]

[(1,
  {'v': 'variable', 'gamma': 'low', 's': 'low', 'tau': 'low'},
  {'t0': [0],
   'v': [1.5e-05],
   'gamma': [-0.8],
   's': [0.0047],
   'tau': [0.21500000000000002]}),
 (2,
  {'v': 'variable', 'gamma': 'low', 's': 'low', 'tau': 'low'},
  {'t0': [0],
   'v': [2.9999999999999997e-05],
   'gamma': [-0.8],
   's': [0.0047],
   'tau': [0.21500000000000002]})]

In [9]:
!mkdir results/parameter_recovery
!mkdir results/parameter_recovery/metropolis
!mkdir results/parameter_recovery/metropolis/partial

mkdir: results/parameter_recovery: File exists
mkdir: results/parameter_recovery/metropolis: File exists
mkdir: results/parameter_recovery/metropolis/partial: File exists


In [10]:
from os.path import join, isfile

In [11]:
def wrap_recover_glam(generated_input):
    
    from pymc3 import summary
    
    index, level_set, parameter_set = generated_input

    # check if already done:
    filename = join('results', 'parameter_recovery', 'metropolis', 'partial', 'parameter_recovery_metropolis_part{}.csv'.format(index))
    if isfile(filename):
        print("Found previous result for index {}. Skipping...".format(index))
        return
    else:
        # perform actual recovery
        result = recover_glam(parameter_set, n_trials=100, n_items=3, seed=index)

        # check for convergence:
        all_converged = summary(result.trace[0], varnames=['v', 'gamma', 's', 'tau'])['Rhat'].between(0.9, 1.1).all()

        # assemble output dataframe row
        output = pd.DataFrame(dict(index=[index],
                                   v_level=[level_set['v']],
                                   gamma_level=[level_set['gamma']],
                                   s_level=[level_set['s']],
                                   tau_level=[level_set['tau']],
                                   v_gen=parameter_set['v'],
                                   gamma_gen=parameter_set['gamma'],
                                   s_gen=parameter_set['s'],
                                   tau_gen=parameter_set['tau'],
                                   v_rec=result.estimates['v'][0],
                                   gamma_rec=result.estimates['gamma'][0],
                                   s_rec=result.estimates['s'][0],
                                   tau_rec=result.estimates['tau'][0],
                                   converged=[all_converged]))

        output.to_csv(filename,
                      index=False)

        return output

In [12]:
wrap_recover_glam(all_generated_inputs[0])

Generating single subject models for 1 subjects...
Fitting 1 model(s) using MCMC...
  Fitting model 1 of 1...


Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
The estimated number of effective samples is smaller than 200 for some parameters.


/!\ Automatically setting parameter precision...


,index,v_level,gamma_level,s_level,tau_level,v_gen,gamma_gen,s_gen,tau_gen,v_rec,gamma_rec,s_rec,tau_rec,converged
0,1,variable,low,low,low,0.000015,-0.8,0.0047,0.215,0.000016,-0.53,0.004534,0.19,True


## Run the recovery in parallel

In [13]:
from multiprocessing import Pool

n_cores = 4
p = Pool(n_cores)

output = p.map(wrap_recover_glam, generate_parameter_sets(parameter_info))

Found previous result for index 1. Skipping...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Fitting 1 model(s) using MCMC...
  Fitting model 1 of 1...


Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
INFO (theano.gof.compilelock): Waiting for existing lock by process '16747' (I am process '16738')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '16747' (I am process '16746')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '16747' (I am process '16742')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '16747' (I am process '16

Fitting 1 model(s) using MCMC...
  Fitting model 1 of 1...


Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
INFO (theano.gof.compilelock): Waiting for existing lock by process '16738' (I am process '16746')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '16738' (I am process '16742')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '16738' (I am process '16748')
INFO (theano.gof.compilelock): Waiting for existing lock by process '16738' (I am process '16744')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.5-64/loc

Fitting 1 model(s) using MCMC...
  Fitting model 1 of 1...
Fitting 1 model(s) using MCMC...
  Fitting model 1 of 1...


...

/!\ Automatically setting parameter precision...
Generating single subject models for 1 subjects...
Fitting 1 model(s) using MCMC...
  Fitting model 1 of 1...


The estimated number of effective samples is smaller than 200 for some parameters.


/!\ Automatically setting parameter precision...


Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
The number of effective samples is smaller than 10% for some parameters.


/!\ Automatically setting parameter precision...


In [14]:
from os import listdir

In [15]:
filenames = [file for file in listdir(join('results', 'parameter_recovery', 'metropolis', 'partial'))
             if file.endswith('.csv')]

partial_recoveries = []

for file in filenames:
    partial = pd.read_csv(join('results', 'parameter_recovery', 'metropolis', 'partial', file))
    partial_recoveries.append(partial)

parameter_recovery = pd.concat(partial_recoveries).sort_values('index').reset_index(drop=True)
del partial_recoveries
parameter_recovery.to_csv(join('results', 'parameter_recovery', 'metropolis', 'parameter_recovery_metropolis.csv'))
parameter_recovery.head()

,index,v_level,gamma_level,s_level,tau_level,v_gen,gamma_gen,s_gen,tau_gen,v_rec,gamma_rec,s_rec,tau_rec,converged
0,1,variable,low,low,low,0.000015,-0.8,0.0047,0.215,0.000016,-0.53,0.004534,0.19,True
1,2,variable,low,low,low,0.000030,-0.8,0.0047,0.215,0.000031,-0.82,0.004665,0.20,True
2,3,variable,low,low,low,0.000045,-0.8,0.0047,0.215,0.000047,-0.71,0.004576,0.27,True
3,4,variable,low,low,low,0.000060,-0.8,0.0047,0.215,0.000062,-0.66,0.004197,0.35,True
4,5,variable,low,low,low,0.000075,-0.8,0.0047,0.215,0.000080,-0.69,0.004514,0.29,True
